# LAB3: Sparsity
Authors: 

    Mathurin Massias (mathurin.massias@gmail.com)
    
    Giacomo Meanti  

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from scipy.io import loadmat

from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split

from lab3_utils import create_random_data

# Warm-up part

## Dataset generation and model fitting

In [ ]:
# what does create_random_data do ?
print(create_random_data.__doc__)

In [ ]:
def train_test_data(n_samples, n_features, n_informative_features, 
                    noise_level):
    """Util function to generate and split random data.
    See the docstring of create_random_data for more details.
    """
    X, y = create_random_data(n_samples, n_features, n_informative_features, 
                              noise_level=noise_level)
    print("X shape:", X.shape)
    print("y shape:", y.shape)
    train_size = 0.8  # proportion of dataset used for training
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, shuffle=False, train_size=train_size)
    return X_train, X_test, y_train, y_test

### Noiseless data

First, generate data without noise: $y = X \beta^\star$, where $\beta^\star$ has `n_informative_features` non-zero entries and $X$ has shape (n_samples, n_features).

We split it into testing and training parts.

In [ ]:
n_samples = 100
n_features = 200
n_informative_features = 50

# first, use noiseless data, split in train and test/validation parts
X_train, X_test, y_train, y_test = train_test_data(
    n_samples, n_features, n_informative_features, noise_level=0.)
print("Training dataset shape:", X_train.shape)
print("Testing dataset shape:", X_test.shape)

In sklearn, the objective function of the `ElasticNet` optimization is:
$$\frac{1}{2 \times \text{n_samples}} \Vert y - X \beta \Vert_2^2 + \alpha \times \left( \text{l1_ratio} \times \Vert \beta \Vert_1 + \frac{1 - \text{l1_ratio}}{2} \Vert \beta \Vert_2^2\right)$$
you can get pure L1 regularization (the LASSO estimator) by setting `l1_ratio` to $1$, and pure L2 regularization (ridge regression) by setting it to $0$.

The datafit normalisation by `n_samples`, frequently used in statistics, is here to make `alpha` insensitive to the number of samples, in a CV setting for example.

For more information, read the docstring after displaying it in the next cell (you can close the documentation popup afterwards by clicking on the cross or hitting Esc).

In [ ]:
ElasticNet?

In [ ]:
# instantiate a classifier with arbitrary values for L1 and L2 penalization
clf = ElasticNet(alpha=0.01, l1_ratio=0.5)

In [ ]:
# fit the model and print its first coefficients
# beware that sklearn fits an intercept by default
clf.fit(X_train, y_train)
print("50 first coefficients of estimated w:\n", clf.coef_[:50])
print("Intercept: %f" % clf.intercept_)
print("Nonzero coefficients: %d out of %d" % ((clf.coef_ != 0.).sum(), clf.coef_.shape[0]))
print("Training error: %.2e" % np.mean((y_train - clf.predict(X_train)) ** 2))
# TODO compute testing error on left out data
# print("Testing error: %.4f" % )
# TODO bonus: why is clf.predict(X_train) not equal to X_train @ clf.coef_? 

For a fixed $\alpha$, test the influence of l1_ratio on the sparsity of the solution and on the behaviors of the train and test errors:

In [ ]:
l1_ratios = [0., 0., 0., 0., 0.]  # TODO choose your own values between 0 and 1

train_errs = np.zeros(len(l1_ratios))
test_errs = np.zeros_like(train_errs)
sparsity = np.zeros_like(train_errs)

for i, l1_ratio in enumerate(l1_ratios):
    clf = # TODO; you may need to tune alpha a bit too.
    # TODO fit the model on train data
    # TODO compute train and test errors
    train_errs[i] = 
    test_errs[i] = 
    sparsity[i] = # number of non-zero elements in clf.coef_
    
plt.figure()
plt.plot(l1_ratios, test_errs, label='Test error')
plt.plot(l1_ratios, train_errs, label='Train error')
plt.xlabel("l1_ratio")
plt.legend()

plt.figure()
plt.plot(l1_ratios, sparsity)
plt.ylabel(r'$||w||_0$')
plt.xlabel('l1_ratio')

In [ ]:
# TODO do the same for the influence of alpha, with a fixed l1_ratio
# What happens when alpha becomes too big?
alphas = np.geomspace(1e-4, 1e4, num=50)
print(alphas)
# TODO plot train/test curve, sparsity curve

### Noisy data
Do the same analysis as above, this time when the observations $y$ are corrupted by additive Gaussian noise:
$$y = X \beta^\star + \varepsilon$$

In [ ]:
# TODO check again the influence of regularization when there is noise in the data.
noise_level = 0.5

X_train, X_test, y_train, y_test = train_test_data(
    n_samples, n_features, n_informative_features, 
    noise_level=noise_level)
# TODO: plot train/test curve and sparsity curve

### Bonus 
An alternative formulation/parametrization of the ElasticNet objective is:
$$\frac{1}{2 \times \text{n_samples}} \Vert y - X \beta \Vert_2^2 + a \Vert \beta \Vert_1 + \frac{b}{2} \Vert \beta \Vert_2^2$$

Express $\alpha$ and l1_ratio as functions of $a$ and $b$.

For a fixed value of $a$, fit the model with increasing values of $b$. How is the sparsity of the solutions affected?  

In [ ]:
# TODO how is the sparsity affected if you increase the L2 regularization ?
a = 
b_values = []
sparsity = np.zeros(len(b_values))
for i, b in enumerate(b_values):
    alpha = 
    l1_ratio = 
    clf = 
    clf.fit(X_train, y_train)
    sparsity[i] = 
# TODO plot

### Influence of dataset size
Observe that the datafitting term is normalized by n_samples, hence it should not grow when the dataset becomes taller (`n_features` is fixed, `n_samples` increases).

Vary one of `n_samples`, `n_features` and `n_informative_features` to observe their influence on the model. What happens when `n_samples` becomes greater that `n_informative_features` ?

In [ ]:
n_samples = # TODO
n_features = # TODO
n_informative_features = # TODO

X_train, X_test, y_train, y_test = train_test_data(
    n_samples, n_features, n_informative_features, noise_level=0.5)

# TODO

## Parameter selection with cross validation
In the next section, we use scikit-learn's built in functions to perform cross validated selection of alpha and l1_ratio.

In [ ]:
X_train, X_test, y_train, y_test = train_test_data(
    n_samples=100, n_features=300, n_informative_features=20, noise_level=1)
# using 3-fold cross validation
clf = ElasticNetCV(l1_ratio=[.1, .4, .8, .99,], cv=3)
clf.fit(X_train, y_train)

In [ ]:
# displaying mean squared errors for various values of l1_ratio
# (values are over CV folds, thick black line is average over folds)
fig, axarr = plt.subplots(2, 2, figsize=(15, 10), 
                          constrained_layout=True, sharey=True)
for i, l1_ratio in enumerate(clf.l1_ratio):
    mse = clf.mse_path_[i]
    alphas = clf.alphas_[i]
    axarr.flat[i].semilogx(alphas, mse, ':')
    axarr.flat[i].semilogx(alphas, mse.mean(axis=-1), 'k',
                 label='Average across the folds', linewidth=2)

    axarr.flat[i].set_xlabel(r'$\alpha$')
    axarr.flat[i].set_ylabel('Mean square error')
    axarr.flat[i].set_title('l1_ratio=%s' % l1_ratio)

In [ ]:
print("Optimal values for l1_ratio and alpha: %s, %.2e" % (clf.l1_ratio_, clf.alpha_))

How does the best l1_ratio evolve when n_informative_features increases ? Why ? 

# Intermediate part

## Classification data

Load some data verifying $y = \text{sign}(X \beta^\star)$ where $\beta^\star$ is unknown and $s$-sparse - but you do not know $s$. **The goal of this part is to infer $s$**.

Note that this problem is notoriously difficult, even in this noiseless case, and cannot be solved without stringent assumptions on the design matrix $X$.

In [ ]:
data = loadmat("../../data/part3-data.mat")

In [ ]:
X = data["X"]
y = data["Y"][:, 0]
print(X.shape, y.shape)
# TODO check numerically that y only contains values equal to 1 or -1

Now you must infer $s$.
A first possible approach is to use the Cross-Validation procedure used in the previous part: find the sparsity of the optimal $\beta$ obtained by cross-validation on a grid of values for $\alpha$ and l1_ratio.

In [ ]:
# TODO find optimal s from a CV point of view

Another way to try to estimate $s$ is to measure the correlation between
the columns of $X$ and $y$. Indeed, the zero coefficients in $\beta^\star$ will ignore the
corresponding columns in $X$ while generating $y$. Can you also identify the indices of these features ?

In [ ]:
# TODO compute correlation
# corr = 

In [ ]:
# sort:
idx = np.argsort(corr)
plt.plot(corr[idx[::-1]])

In [ ]:
# TODO identify the cutoff numerically, get s 
# and the indices of highest correlated features
# highly_corr_feats =

Finally, use again the code of the first part, and tune the sparsity parameter l1_ratio so that
it selects only $s$ features ($s$ being your sparsity estimate from the previous
question). Look at which are the selected features in your solution. Do they
correspond to the ones you identified with the correlation approach? 
If they do not, can you figure out why does this happen?